# `l2hmc`-qcd

## Imports / Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
#tf.keras.backend.set_floatx()
import horovod.tensorflow as hvd
hvd.init()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    gpu = gpus[hvd.local_rank()]
    tf.config.experimental.set_visible_devices(gpu, 'GPU')
    
import numpy as np
import torch

In [3]:
from l2hmc.lattice.su3.pytorch.lattice import LatticeSU3 as LatticeSU3pt
from l2hmc.lattice.su3.tensorflow.lattice import LatticeSU3 as LatticeSU3tf

import l2hmc.group.pytorch.group as gpt
import l2hmc.group.tensorflow.group as gtf

2022-06-05 06:17:44.960566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
from __future__ import absolute_import, print_function, annotations, division
import os
import hydra

from typing import Optional
from pathlib import Path

from omegaconf import OmegaConf
from hydra import (
    initialize,
    initialize_config_module,
    initialize_config_dir,
    compose
)

from l2hmc.common import get_timestamp
from l2hmc.configs import CONF_DIR
from l2hmc.experiment import Experiment
from hydra.core.global_hydra import GlobalHydra

from l2hmc.utils.rich import print_config

## Instantiate Configs

1. Initialize [`Hydra`](https://hydra.cc) by registering `CONF_DIR` as `config_dir`
2. Compose `conf/config.yaml` with user-specified `overrides` to create `DictConfig` for `Experiment`

In [5]:
from l2hmc.utils.rich import print_config
from l2hmc.experiment import Experiment
import l2hmc.group.pytorch.group as gpt
import l2hmc.group.tensorflow.group as gtf
#from hydra.core.hydra_config import HydraConfig

def get_experiment(overrides: list[str] = None):
    GlobalHydra.instance().clear()
    with initialize_config_dir(CONF_DIR.absolute().as_posix()):
        cfg = compose('config', overrides=overrides)
        
    return Experiment(cfg)


def train(overrides: list[str] = None):
    GlobalHydra.instance().clear()
    with initialize_config_dir(CONF_DIR.absolute().as_posix()):
        cfg = compose('config', overrides=overrides)
        #print(cfg.hydra.runtime.output_dir)
        #print(f'Output_dir = {HydraConfig.get().runtime.output_dir}')
        if cfg.framework == 'pytorch':
            from l2hmc.scripts.pytorch.main import main as main_pt
            return main_pt(cfg)
        elif cfg.framework == 'tensorflow':
            from l2hmc.scripts.tensorflow.main import main as main_tf
            return main_tf(cfg)

        raise ValueError('Expected `cfg.framework` to be one of "pytorch", "tensorflow".')

In [17]:
GlobalHydra.instance().clear()
defaults = [
    'mode=debug',
    'steps.nera=0',
    'steps.nepoch=0',
    'steps.test=1000',
    'eps_hmc=0.001',
    'dynamics=su3',
    'dynamics.nchains=64',
    'dynamics.nleapfrog=8',
    'dynamics.latvolume=[4, 4, 4, 16]',
    'annealing_schedule.beta_init=6.0',
    'annealing_schedule.beta_final=6.0',
    'wandb.setup.sync_tensorboard=false',
]
exp_pt = get_experiment(overrides=[
    *defaults,
    'framework=pytorch',
])

exp_tf = get_experiment(overrides=[
    *defaults,
    'framework=tensorflow',
])

objs_pt = exp_pt.build(init_wandb=False)
objs_tf = exp_tf.build(init_wandb=False)

trainer_pt = exp_pt.trainer
dynamics_pt = exp_pt.dynamics
lattice_pt = exp_pt.lattice

trainer_tf = exp_tf.trainer
dynamics_tf = exp_tf.dynamics
lattice_tf = exp_tf.lattice

xpt = lattice_pt.random()
xtf = tf.constant(xpt.detach().numpy())

xpt.shape
(xpt.numpy() - xtf.numpy()).sum()

xdim: 32768
group: SU3
xshape: (64, 4, 4, 4, 4, 16, 3, 3)
latvolume: [4, 4, 4, 16]
xdim: 32768
group: SU3
xshape: (64, 4, 4, 4, 4, 16, 3, 3)
latvolume: [4, 4, 4, 16]


torch.Size([64, 4, 4, 4, 4, 16, 3, 3])

0j

In [18]:
from l2hmc.distributions.pytorch.haarSUN import HaarSUN as HaarSUNpt
hSU3 = HaarSUNpt(3)

In [19]:
nsamples = lattice_pt.nlinks * dynamics_pt.config.nchains
zpt = hSU3.rsample(nsamples)
zpt = zpt.reshape_as(xpt)
zpt.shape

torch.Size([64, 4, 4, 4, 4, 16, 3, 3])

In [20]:
z = hSU3.rsample(1)
z.shape

torch.Size([1, 3, 3])

In [21]:
z

tensor([[[-0.0052-0.0233j, -0.0364+0.0879j, -1.7248-0.2367j],
         [ 0.9736-0.4418j,  0.3871+0.6046j, -0.0729-0.0018j],
         [-0.1766+0.7463j, -1.0287-0.1943j, -0.0377-0.0266j]]])

In [23]:
zHz = (z.mH @ z)[0]
zHz.det()

tensor(1.-0.j)

In [24]:
gpt.checkSU(z)
gpt.checkU(z)

(tensor(0.8398), tensor(0.8398))

(tensor(0.7109+0.j), tensor(0.7109+0.j))

## Run HMC with $SU(3)$

In [25]:
from rich.console import Console

console = Console(force_jupyter=True, log_path=False)

def run_hmc(
    experiment: Experiment,
    beta: float = 1.0,
    nlog: int = 10,
    nsteps: Optional[int] = None,
    x: Optional[torch.Tensor | tf.Tensor] = None,
) -> dict:
    trainer = experiment.trainer
    dynamics = experiment.dynamics
    lattice = experiment.lattice
    if x is None:
        x = trainer.g.random(lattice._shape)
    #x = lattice.random()
    if isinstance(x, torch.Tensor):
        beta = torch.tensor(beta)
    elif isinstance(x, tf.Tensor):
        beta = tf.constant(beta)
        
    nsteps = (
        experiment.cfg.steps.test
        if nsteps is None else nsteps
    )
    metrics = {}
    for step in range(nsteps):
        x, metrics_ = experiment.dynamics.apply_transition_hmc(
            (x, beta), eps=experiment.cfg.eps_hmc
        )
        _ = metrics_.pop('mc_states')
        
        if step % nlog == 0:
            mstrs = []
            lmetrics = lattice.calc_metrics(x=x)
            for key, val in metrics_.items():
                if isinstance(x, torch.Tensor):
                    mstrs.append(
                        f'{key}: {val.detach().numpy().mean():.4f}'
                    )
                elif isinstance(x, tf.Tensor):
                    mstrs.append(
                        f'{key}: {val.numpy().mean():.4f}'
                    )

                try:
                    metrics[key].append(val)
                except KeyError:
                    metrics[key] = [val]

            if isinstance(x, torch.Tensor):
                lmstrs = [
                    f'{k}: {v.detach().numpy().mean():.4f}'
                    for k, v in lmetrics.items()
                ]
            elif isinstance(x, tf.Tensor):
                lmstrs = [
                    f'{k}: {v.numpy().mean():.4f}'
                    for k, v in lmetrics.items()
                ]
                
            console.log(', '.join(
                [f'{step} / {experiment.cfg.steps.test}', *mstrs, *lmstrs]
            ))
    return metrics

In [27]:
run_hmc(exp_pt, beta=6.0, x=xpt, nsteps=1000, nlog=1)

[06:38:52] 0 / 1000, energy: 17.4181, logprob: 17.4181, logdet: 0.0000, acc: 0.0000,         
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:39:10] 1 / 1000, energy: -24.7137, logprob: -24.7137, logdet: 0.0000, acc: 0.0000,       
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:39:26] 2 / 1000, energy: 20.0416, logprob: 20.0416, logdet: 0.0000, acc: 0.0000,         
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:39:41] 3 / 1000, energy: 22.3528, logprob: 22.3528, logdet: 0.0000, acc: 0.0000,         
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:40:00] 4 / 1000, energy: 2.9820, logprob: 2.9820, logdet: 0.0000, acc: 0.0000, sumlogdet:
           0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:40:18] 5 / 1000, energy: 30.5779, logprob: 30.5779, logdet: 0.0000, acc: 0.0000,         
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:40:36] 6 / 1000, energy: -2.0000, logprob: -2.0000, logdet: 0.0000, acc: 0.0000,         
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

[06:40:57] 7 / 1000, energy: 19.2119, logprob: 19.2119, logdet: 0.0000, acc: 0.0000,         
           sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 0.0002, intQ: 0.0140, plaqs: 0.0001

KeyboardInterrupt: 

In [12]:
run_hmc(exp_pt)

[20:56:31] 0 / 1000, energy: 0.4590, logprob: 0.4590, logdet: 0.0000, acc: 1.0000, sumlogdet:
           0.0000, acc_mask: 1.0000, sinQ: -0.0019, intQ: -0.0277, plaqs: 0.0028

[20:56:44] 20 / 1000, energy: -38.0777, logprob: -38.0777, logdet: 0.0000, acc: 1.0000,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0014, intQ: 0.0209, plaqs: 0.0203

[20:56:54] 40 / 1000, energy: -15.9243, logprob: -15.9243, logdet: 0.0000, acc: 1.0000,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0015, intQ: 0.0215, plaqs: 0.0371

[20:57:05] 60 / 1000, energy: -13.1612, logprob: -13.1612, logdet: 0.0000, acc: 1.0000,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: -0.0019, intQ: -0.0270, plaqs: 0.0585

[20:57:16] 80 / 1000, energy: -117.2990, logprob: -117.2990, logdet: 0.0000, acc: 1.0000,    
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: -0.0005, intQ: -0.0078, plaqs: 0.0826

...

[21:07:06] 900 / 1000, energy: -272736.0938, logprob: -272736.0938, logdet: 0.0000, acc:     
           0.0006, sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 2.5535, intQ: 37.2558, plaqs:  
           177.5215

[21:07:27] 920 / 1000, energy: -272674.0938, logprob: -272674.0938, logdet: 0.0000, acc:     
           0.0006, sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 2.5535, intQ: 37.2558, plaqs:  
           177.5215

[21:07:48] 940 / 1000, energy: -272621.5625, logprob: -272621.5625, logdet: 0.0000, acc:     
           0.0006, sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 2.5535, intQ: 37.2558, plaqs:  
           177.5215

[21:08:11] 960 / 1000, energy: -272664.2812, logprob: -272664.2812, logdet: 0.0000, acc:     
           0.0006, sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 2.5535, intQ: 37.2558, plaqs:  
           177.5215

[21:08:33] 980 / 1000, energy: -272707.6250, logprob: -272707.6250, logdet: 0.0000, acc:     
           0.0006, sumlogdet: 0.0000, acc_mask: 0.0000, sinQ: 2.5535, intQ: 37.2558, plaqs:  
           177.5215

In [23]:
run_hmc(exp_pt, beta=6.0, x=xpt)

[22:56:50] 0 / 1000, energy: -82.4791, logprob: -82.4791, logdet: 0.0000, acc: 0.9729,       
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0016, intQ: 0.0238, plaqs: 0.0047

[22:57:01] 20 / 1000, energy: -92.5592, logprob: -92.5592, logdet: 0.0000, acc: 0.8232,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0015, intQ: 0.0223, plaqs: 0.0048

[22:57:09] 40 / 1000, energy: -34.4372, logprob: -34.4372, logdet: 0.0000, acc: 0.7840,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0014, intQ: 0.0210, plaqs: 0.0049

[22:57:16] 60 / 1000, energy: -52.6966, logprob: -52.6966, logdet: 0.0000, acc: 0.9316,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0015, intQ: 0.0212, plaqs: 0.0050

[22:57:23] 80 / 1000, energy: -69.0476, logprob: -69.0476, logdet: 0.0000, acc: 0.9717,      
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0014, intQ: 0.0209, plaqs: 0.0050

...

[23:03:07] 900 / 1000, energy: -94.8275, logprob: -94.8275, logdet: 0.0000, acc: 0.8409,     
           sumlogdet: 0.0000, acc_mask: 1.0000, sinQ: 0.0006, intQ: 0.0094, plaqs: 0.0082

[23:03:15] 920 / 1000, energy: -140.5410, logprob: -140.5410, logdet: 0.0000, acc: 0.8500,   
           sumlogdet: 0.0000, acc_mask: 0.4000, sinQ: 0.0007, intQ: 0.0097, plaqs: 0.0083

[23:03:24] 940 / 1000, energy: -10.3552, logprob: -10.3552, logdet: 0.0000, acc: 0.7166,     
           sumlogdet: 0.0000, acc_mask: 0.8000, sinQ: 0.0007, intQ: 0.0104, plaqs: 0.0083

[23:03:32] 960 / 1000, energy: -76.9752, logprob: -76.9752, logdet: 0.0000, acc: 0.6380,     
           sumlogdet: 0.0000, acc_mask: 0.6000, sinQ: 0.0006, intQ: 0.0094, plaqs: 0.0084

[23:03:40] 980 / 1000, energy: -107.8878, logprob: -107.8878, logdet: 0.0000, acc: 0.5969,   
           sumlogdet: 0.0000, acc_mask: 0.4000, sinQ: 0.0007, intQ: 0.0097, plaqs: 0.0085

In [24]:
run_hmc(exp_tf, beta=6.0, x=xtf)

TypeError: `x` and `y` must have the same dtype, got tf.complex64 != tf.int32.

In [25]:
%debug

> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/tensorflow/python/ops/math_ops.py(1516)_truediv_python3()
   1514     y_dtype = y.dtype.base_dtype
   1515     if x_dtype != y_dtype:
-> 1516       raise TypeError(f"`x` and `y` must have the same dtype, "
   1517                       f"got {x_dtype!r} != {y_dtype!r}.")
   1518     try:

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/tensorflow/python/util/traceback_utils.py(153)error_handler()
    151     except Exception as e:
    152       filtered_tb = _process_traceback_frames(e.__traceback__)
--> 153       raise e.with_traceback(filtered_tb) from None
    154     finally:
    155       del filtered_tb

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/src/l2hmc/group/tensorflow/group.py(417)exp()
    415 def exp(m: Tensor, order: int = 12):
    416     eye = eyeOf(m)
--> 417     x = eye + m / tf.constant(order)
    418     for i in tf.range(order-1, 0, -1):
    419         x = e

In [17]:
xpt[0].det()

tensor([[[[[3.9512e+00+1.1921e-06j, 1.9212e-01-1.4901e-08j,
            6.6092e+00+2.2650e-06j, 2.2727e+00+3.5167e-06j],
           [2.5074e+00-6.5565e-07j, 3.4343e+01-7.1526e-06j,
            8.2308e+00+0.0000e+00j, 3.8428e+00+1.1921e-07j],
           [3.2881e+00-3.5763e-07j, 8.3902e+00-3.5763e-06j,
            1.6303e+01-1.9073e-06j, 2.5934e+01-3.3379e-06j],
           [1.4891e+00+1.4901e-06j, 4.8264e+00-4.7684e-07j,
            4.3608e+00+1.1921e-07j, 3.4727e+01-1.9073e-06j]],

          [[1.5873e+01+4.5300e-06j, 6.1351e+00+8.3447e-07j,
            5.3375e+01-3.2425e-05j, 9.4761e-01+4.1723e-07j],
           [1.1289e+01+9.5367e-07j, 4.1570e+01-3.0518e-05j,
            1.7626e+00-1.0729e-06j, 1.0562e+00+1.7509e-07j],
           [2.9871e-02+1.8626e-09j, 1.9787e+01-0.0000e+00j,
            4.1281e+01-9.5367e-06j, 2.1909e+00+7.1526e-07j],
           [6.8007e+00+9.5367e-07j, 5.7136e+00+9.5367e-07j,
            4.3490e+00-3.5763e-07j, 7.5869e-01+0.0000e+00j]],

          [[1.1828e+01+4.053

In [15]:
inputs_pt = (xpt, torch.tensor(1.0))
xpt_, hmetrics_pt = exp_pt.dynamics.apply_transition_hmc(inputs_pt, eps=torch.tensor(0.00001))

In [12]:
inputs_tf = (xtf, tf.constant(1.0))
xtf_, hmetrics_tf = exp_tf.dynamics.apply_transition_hmc(inputs_tf, eps=tf.constant(0.000001))

In [13]:
(xpt.numpy() - xtf.numpy()).sum()

0j

In [24]:
mcstates_pt = hmetrics_pt.pop('mc_states', None)
mcstates_tf = hmetrics_tf.pop('mc_states', None)
print('\n'.join([f'{key} (pt): {val.mean()}' for key, val in hmetrics_pt.items()]))
print(80 *'-')
print('\n'.join([f'{key} (tf): {val.numpy().mean()}' for key, val in hmetrics_tf.items()]))

energy (pt): 31637.9921875
logprob (pt): 31637.9921875
logdet (pt): 0.0
acc (pt): 0.9787447452545166
sumlogdet (pt): 0.0
acc_mask (pt): 1.0
--------------------------------------------------------------------------------
energy (tf): 31656.37890625
logprob (tf): 31656.37890625
logdet (tf): 0.0
acc (tf): 0.8470330238342285
sumlogdet (tf): 0.0
acc_mask (tf): (1+0j)


In [25]:
lmetrics_pt = lattice_pt.calc_metrics(x=xpt)

In [26]:
lmetrics_tf = lattice_tf.calc_metrics(x=xtf)

In [27]:
wlpt, _ = lattice_pt._wilson_loops(xpt, needs_rect=False)
wltf, _ = lattice_tf._wilson_loops(xtf, needs_rect=False)

In [28]:
dwl = (wlpt.numpy() - wltf.numpy())

In [29]:
(dwl ** 2).sum()

0j

In [30]:
qpt = wlpt.imag.sum(tuple(range(2, len(wlpt.shape)))).sum(0)
qtf = tf.reduce_sum(
    tf.reduce_sum(tf.math.imag(wltf), axis=range(2, len(wltf.shape))),
    axis=0
)
qpt /= (32 * (np.pi ** 2))
qtf /= (32 * (np.pi ** 2))

In [31]:
qpt.numpy()

array([269.0069], dtype=float32)

In [32]:
qtf.numpy()

array([269.0069], dtype=float32)

In [33]:
((qpt.numpy() - qtf.numpy()) ** 2).sum()

0.0

In [37]:
xpt_ = lattice_pt.g.compat_proj(xpt)
xtf_ = lattice_tf.g.compat_proj(xtf)

In [46]:
xpt_[0, 0, 0, 0, 0, 0].numpy()
xtf_[0, 0, 0, 0, 0, 0].numpy()

array([[ -18.315264 -35.850132j, -410.2562  +170.15718j ,
          15.325998 +98.029655j],
       [   8.945226 +68.93815j ,  766.6843   -38.112377j,
          32.19337 -168.39961j ],
       [  41.667877 +35.655823j,  428.824   -426.66763j ,
         -69.96763 -115.61467j ]], dtype=complex64)

array([[ -18.301857 -35.82389j , -409.95587 +170.03262j ,
          15.314779 +97.9579j  ],
       [   8.938678 +68.88769j ,  766.1231   -38.084476j,
          32.169807-168.27635j ],
       [  41.637375 +35.629723j,  428.51013 -426.3553j  ,
         -69.91641 -115.53004j ]], dtype=complex64)

In [42]:
xpt_2 = xpt_.adjoint() @ xpt_
xtf_2 = tf.linalg.adjoint(xtf_) @ xtf_

In [43]:
((xpt_2.numpy() - xtf_2.numpy()) ** 2).sum()

(nan+nanj)

In [39]:
((xpt_.numpy() - xtf_.numpy()) ** 2).sum()

(nan+nanj)

In [34]:
diffs = {}
for key, val in lmetrics_pt.items():
    vpt = val.numpy()
    vtf = lmetrics_tf[key].numpy()
    diffs[key] = ((vpt - vtf) ** 2).sum()
    
print(f'\n'.join([f'{k}: {v}' for k, v in diffs.items()]))

sinQ: 3.637978807091713e-12
intQ: 0.0
plaqs: 3.637978807091713e-12


In [36]:
lattice_pt.g.trace(xpt.adjoint() @ xpt).sum()

tensor(582158.4375+0.j)

In [ ]:
lattice_tf.g.trace(tf.m)

In [60]:
xpt_.shape

torch.Size([4, 9216])

In [69]:
wlpt, _ = lattice_pt._wilson_loops(xpt, needs_rect=False)
wltf, _ = lattice_tf._wilson_loops(xtf, needs_rect=False)

In [70]:
wlpt.shape

torch.Size([6, 4, 4, 4, 4, 4])

In [71]:
pspt = wlpt.real.sum(tuple(range(2, len(wlpt.shape))))
pspt.shape

torch.Size([6, 4])

In [63]:
wltf, _ = lattice_tf._wilson_loops(xtf, needs_rect=False)

In [72]:
(wlpt.numpy() - wltf.numpy()).sum()

0j

In [73]:
qsin_pt = lattice_pt.sin_charges(xpt)
qsin_tf = lattice_tf.sin_charges(xtf)
(qsin_pt.numpy() - qsin_tf.numpy()).sum()

-3.4332275e-05

In [ ]:
qint_pt = lattice_pt.int_charges

In [65]:
plaqs_tf = lattice_tf._plaquettes(xtf)

In [67]:
plaqs_pt = lattice_pt._plaquettes(xpt)

In [68]:
plaqs_pt.shape

torch.Size([4])

In [66]:
plaqs_tf.shape

TensorShape([4])

In [64]:
wltf.shape

TensorShape([6, 4, 4, 4, 4, 4])

In [53]:
ps, _ = lattice_pt._wilson_loops(xpt_)

In [54]:
ps.shape

torch.Size([6, 4, 4, 4, 4, 4])

In [48]:
qsinpt_ = lattice_pt.sin_charges(xpt_)

In [49]:
qintpt_ = lattice_pt.int_charges(xpt_)

In [50]:
qsinpt_.shape

torch.Size([4])

In [52]:
qsinpt_

tensor([  48.5113,   10.5180,   -3.0380, -132.9389], grad_fn=<DivBackward0>)

In [51]:
qintpt_

tensor([  707.7921,   153.4606,   -44.3246, -1939.6115],
       grad_fn=<DivBackward0>)

In [28]:
xpt_.shape

torch.Size([4, 9216])

In [29]:
hmetrics_pt.keys()

dict_keys(['energy', 'logprob', 'logdet', 'acc', 'sumlogdet', 'acc_mask', 'mc_states'])

In [31]:
hmetrics_pt['acc']

tensor([0.9394, 0.9248, 0.9321, 1.0000], grad_fn=<SWhereBackward0>)

In [32]:
hmetrics_pt['sumlogdet']

tensor([0., 0., 0., 0.])

In [33]:
xout, metrics = trainer_pt.hmc_step(inputs_pt, eps=torch.tensor(0.0001))

In [47]:
inputs_tf = (xtf, tf.constant(1.0))
xout_tf, metrics_tf = trainer_tf.hmc_step(inputs_tf, eps=tf.constant(0.0001))

ValueError: in user code:

    File "/Users/saforem2/projects/l2hmc-qcd/src/l2hmc/trainers/tensorflow/trainer.py", line 334, in hmc_step  *
        xo = self.dynamics.g.compat_proj(xo)
    File "/Users/saforem2/projects/l2hmc-qcd/src/l2hmc/group/tensorflow/group.py", line 133, in compat_proj  *
        return projectSU(x)
    File "/Users/saforem2/projects/l2hmc-qcd/src/l2hmc/group/tensorflow/group.py", line 273, in projectSU  *
        m = projectU(x)
    File "/Users/saforem2/projects/l2hmc-qcd/src/l2hmc/group/tensorflow/group.py", line 267, in projectU  *
        t2 = rsqrtPHM3(t)
    File "/Users/saforem2/projects/l2hmc-qcd/src/l2hmc/group/tensorflow/group.py", line 260, in rsqrtPHM3  *
        return term0 + term1 + term2

    ValueError: Dimensions must be equal, but are 3 and 9216 for '{{node add_53}} = AddV2[T=DT_COMPLEX64](Cast_19, Mul_404)' with input shapes: [3,3], [9216,9216].


In [46]:
xtf_.shape

TensorShape([4, 9216])

In [41]:
xtf_, metrics_tf = dynamics_tf(inputs_tf)

> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/tensorflow/python/framework/ops.py(7164)raise_from_not_ok_status()
   7162 def raise_from_not_ok_status(e, name):
   7163   e.message += (" name: " + name if name is not None else "")
-> 7164   raise core._status_to_exception(e) from None  # pylint: disable=protected-access
   7165 
   7166 

ipdb> l
   7159   return kwarg_only(f)
   7160 
   7161 
   7162 def raise_from_not_ok_status(e, name):
   7163   e.message += (" name: " + name if name is not None else "")
-> 7164   raise core._status_to_exception(e) from None  # pylint: disable=protected-access
   7165 
   7166 
   7167 def add_exit_callback_to_default_func_graph(fn):
   7168   """Add a callback to run when the default function graph goes out of scope.
   7169 

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/tensorflow/python/util/traceback_utils.py(153)error_handler()
    151     except Exception as e:
    152       filter

In [19]:
xtf_, metrics_tf = dynamics_tf(inputs_tf)

ValueError: Exception encountered when calling layer "dynamics" (type Dynamics).

Input 0 of layer "vnet_0" is incompatible with the layer: expected shape=(None, 9216), found shape=(4, 4, 4, 4, 4, 4, 3, 3)

Call arguments received:
  • inputs=('tf.Tensor(shape=(4, 4, 4, 4, 4, 4, 3, 3), dtype=complex64)', 'tf.Tensor(shape=(), dtype=float32)')
  • training=True

In [20]:
%debug

> /Users/saforem2/projects/l2hmc-qcd/src/l2hmc/dynamics/tensorflow/dynamics.py(545)_call_vnet()
    543         vnet = self._get_vnet(step)
    544         assert callable(vnet)
--> 545         return vnet(inputs, training)
    546 
    547     def _get_xnet(self, step: int, first: bool) -> CallableNetwork:

ipdb> x, beta = inputs
ipdb> x.shape
TensorShape([4, 4, 4, 4, 4, 4, 3, 3])
ipdb> xf = x.reshape(x.shape[0], -1)
*** AttributeError: 
        'EagerTensor' object has no attribute 'reshape'.
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()
ipdb> xf = tf.reshape(x, (x.shape[0], -1))
ipdb> xf.shape
TensorShape([4, 9216])
ipdb> vout = vnet((xf, beta), training)
*** ValueError: Input 1 of layer "vnet_0" is incompatible with the layer: expected shape=(None, 9216), found shape=(4, 4, 4, 4, 4, 4, 3, 3)
ipdb> vout = vnet((tf.reshape(x, (x.shape[0], -1)), bet

In [13]:
cfg = compose('config.yaml', overrides=['framework=pytorch'])

ConfigCompositionException: 

In [14]:
%debug

> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/_utils.py(716)_raise()
    714     else:
    715         ex.__cause__ = None
--> 716     raise ex.with_traceback(sys.exc_info()[2])  # set end OC_CAUSE=1 for full backtrace
    717 
    718 

ipdb> q


{}

In [ ]:
GlobalHydra.instance().clear()
conf_dir = CONF_DIR.resolve().absolute().as_posix()
initialize_config_dir(conf_dir)
cfg = compose('config')  # , overrides=overrides)

In [15]:
cfg = get_cfg([])

MissingConfigException: Cannot find primary config 'ExperimentConfig'. Check that it's in your config search path.

Config search path:
	provider=hydra, path=pkg://hydra.conf
	provider=main, path=file:///Users/saforem2/projects/l2hmc-qcd/src/l2hmc/conf
	provider=hydra-colorlog, path=pkg://hydra_plugins.hydra_colorlog.conf
	provider=schema, path=structured://

In [6]:
conf = compose(config_name='config')

ConfigCompositionException: Error merging 'config' with schema

In [14]:
#print_config(OmegaConf.to_object('../conf/config.yaml'))

TypeError: 'NoneType' object is not iterable

In [4]:
latpt = LatticeSU3pt(5, (4, 4, 4, 4))
lattf = LatticeSU3tf(5, (4, 4, 4, 4))

In [5]:
xpt = latpt.random()
xnp = xpt.numpy()
xtf = tf.constant(xnp)

In [7]:
wlpt = latpt.wilson_loops(xpt)
wltf = lattf.wilson_loops(xtf)

In [8]:
(wlpt.numpy() - wltf.numpy()).sum()

0j

In [9]:
plaqs_pt = latpt._plaquettes(xpt)
plaqs_tf = lattf._plaquettes(xtf)

In [10]:
(plaqs_pt.numpy() - plaqs_tf.numpy()).sum()

-3.6001205e-05

In [11]:
qint_pt = latpt._int_charges(wlpt)
qint_tf = lattf._int_charges(wltf)

In [12]:
(qint_pt.numpy() - qint_tf.numpy()).sum()

-0.00024414062

In [13]:
charges_pt = latpt.charges(xpt)
charges_tf = lattf.charges(xtf)

In [14]:
(charges_pt['qint'].numpy() - charges_tf['qint'].numpy()).sum()

-0.00024414062

In [15]:
(charges_pt['qsin'].numpy() - charges_tf['qsin'].numpy()).sum()

-1.9073486e-05

In [16]:
from l2hmc.configs import DynamicsConfig
from l2hmc.dynamics.pytorch.dynamics import Dynamics as ptDynamics
from l2hmc.dynamics.tensorflow.dynamics import Dynamics as tfDynamics

In [17]:
from __future__ import absolute_import, print_function, annotations, division
import os
import hydra

from typing import Optional
from pathlib import Path

from omegaconf import OmegaConf
from hydra import (
    initialize,
    initialize_config_module,
    initialize_config_dir,
    compose
)

from l2hmc.common import get_timestamp
from l2hmc.configs import CONF_DIR
from l2hmc.experiment import Experiment
from hydra.core.global_hydra import GlobalHydra

from l2hmc.utils.rich import print_config


In [35]:
GlobalHydra.instance().clear()
initialize(config_path="../conf/")
cfg = compose('config', overrides=['mode=debug', 'framework=pytorch'])
#conf = OmegaConf.load('../conf/config.yaml')
print_config(conf)

hydra.initialize()

ConfigCompositionException: 

In [36]:
%debug

> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/_utils.py(716)_raise()
    714     else:
    715         ex.__cause__ = None
--> 716     raise ex.with_traceback(sys.exc_info()[2])  # set end OC_CAUSE=1 for full backtrace
    717 
    718 

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/_utils.py(818)format_and_raise()
    816         ex.ref_type_str = ref_type_str
    817 
--> 818     _raise(ex, cause)
    819 
    820 

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/base.py(190)_format_and_raise()
    188         self, key: Any, value: Any, cause: Exception, type_override: Any = None
    189     ) -> None:
--> 190         format_and_raise(
    191             node=self,
    192             key=key,

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/dictconfig.py(166)_validate_get()
    164                 else:
    165      

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/base.py(190)_format_and_raise()
    188         self, key: Any, value: Any, cause: Exception, type_override: Any = None
    189     ) -> None:
--> 190         format_and_raise(
    191             node=self,
    192             key=key,

ipdb> key
'_target_'
ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/dictconfig.py(310)__setitem__()
    308             self.__set_impl(key=key, value=value)
    309         except AttributeError as e:
--> 310             self._format_and_raise(
    311                 key=key, value=value, type_override=ConfigKeyError, cause=e
    312             )

ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/basecontainer.py(392)_map_merge()
    390                         dest[key] = src._get_node(key)
    391                 else:
--> 392                     dest[key] = src._get_node(ke

ipdb> others.keys()
*** AttributeError: 'tuple' object has no attribute 'keys'
ipdb> others[0]
{'_target_': 'l2hmc.configs.Steps', 'nera': 2, 'nepoch': 5, 'test': 50, 'log': 1, 'print': 1}
ipdb> up
> /Users/saforem2/projects/l2hmc-qcd/.venv/lib/python3.10/site-packages/omegaconf/_utils.py(716)_raise()
    714     else:
    715         ex.__cause__ = None
--> 716     raise ex.with_traceback(sys.exc_info()[2])  # set end OC_CAUSE=1 for full backtrace
    717 
    718 

ipdb> q


In [ ]:
GlobalHydra.instance().clear()
initialize(config_path='../conf/') # Assume the configuration file is in the current folder
cfg = compose('config', strict=False)
#cfg = compose('config', return_hydra_config=True)
print_config(cfg)

In [34]:
def get_experiment():
    with initialize_config_dir(CONF_DIR):
        cfg = compose(config_name='config.yaml', overrides=overrides)
        print(OmegaConf.to_yaml(cfg, resolve=True))
        #cfile = outdir.joinpath('config.yaml')
        #assert callable(main)
        #output = main(cfg)
        return Experiment(cfg)

    #print(f'Saving config to: {cfile}')
    #with open(cfile, 'w') as f:
    #    f.write(OmegaConf.to_yaml(cfg, resolve=True))

SyntaxError: 'return' outside function (1456260.py, line 7)

In [46]:

#initialize(config_path='../conf')  # , job_name="test_app")
initialize_config_dir(CONF_DIR.as_posix())
#cfg = compose(config_name="config")  # , overrides=['dynamics=su3', 'framework=pytorch'])
#print_config(cfg)
#print(OmegaConf.to_yaml(cfg))
#cfg = compose(config_name='config.yaml')

hydra.initialize_config_dir()

In [57]:
GlobalHydra.instance().clear()
#with initialize(config_path="../conf/"):
with initialize_config_dir(CONF_DIR.as_posix()):
    cfg = compose(config_name='config') #, overrides=['mode=debug', 'framework=pytorch'])
    print(OmegaConf.to_yaml(cfg))
    #experiment = Experiment(cfg)

ConfigCompositionException: Error merging 'config' with schema

In [60]:
from l2hmc.configs import ExperimentConfig
schema = OmegaConf.structured(ExperimentConfig)
conf = OmegaConf.load("../conf/config.yaml")
#with raises(ValidationError):
cfg = OmegaConf.merge(schema, conf)

ConfigKeyError: Key '_target_' not in 'ExperimentConfig'
    full_key: _target_
    object_type=ExperimentConfig

In [63]:
from hydra import initialize, compose
GlobalHydra.instance().clear()
initialize(config_path='../') # Assume the configuration file is in the current folder
cfg = compose()
#cfg = compose(config_name='config', overrides=['framework=pytorch', 'mode=debug'])

hydra.initialize()

In [47]:
cfg = compose(config_name='config.yaml')  # , overrides=['dynamics=su3', 'framework=pytorch'])

ConfigCompositionException: Error merging 'config.yaml' with schema

In [33]:
from omegaconf import DictConfig, OmegaConf

cfg = OmegaConf.load(CONF_DIR.joinpath('config.yaml').as_posix())
cfg = DictConfig()

{'_target_': 'l2hmc.configs.ExperimentConfig', 'framework': '???', 'profile': False, 'precision': 'float32', 'width': 235, 'eps_hmc': 0.1181, 'compile': True, 'nchains': 128, 'ignore_warnings': True, 'name': None, 'defaults': ['_self_', {'steps': 'default.yaml'}, {'dynamics': 'default.yaml'}, {'wandb': 'default.yaml'}, {'logdir': 'default.yaml'}, {'loss': 'default.yaml'}, {'network': 'default.yaml'}, {'conv': 'default.yaml'}, {'net_weights': 'default.yaml'}, {'learning_rate': 'default.yaml'}, {'annealing_schedule': 'default.yaml'}, {'mode': 'default.yaml'}, {'experiment': None}, {'override hydra/hydra_logging': 'colorlog'}, {'override hydra/job_logging': 'colorlog'}]}